In [5]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont

In [4]:
def display_image(bucket,photo,response):
    # Load image from S3 bucket
    s3_connection = boto3.resource('s3')

    s3_object = s3_connection.Object(bucket,photo)
    s3_response = s3_object.get()

    stream = io.BytesIO(s3_response['Body'].read())
    image=Image.open(stream)

    # Ready image to draw bounding boxes on it.
    imgWidth, imgHeight = image.size
    draw = ImageDraw.Draw(image)

    # calculate and display bounding boxes for each detected custom label
    print('Detected custom labels for ' + photo)
    for customLabel in response['CustomLabels']:
        print('Label ' + str(customLabel['Name']))
        print('Confidence ' + str(customLabel['Confidence']))
        if 'Geometry' in customLabel:
            box = customLabel['Geometry']['BoundingBox']
            left = imgWidth * box['Left']
            top = imgHeight * box['Top']
            width = imgWidth * box['Width']
            height = imgHeight * box['Height']

            fnt = ImageFont.truetype('/Library/Fonts/Arial.ttf', 50)
            draw.text((left,top), customLabel['Name'], fill='#00d400', font=fnt)

            print('Left: ' + '{0:.0f}'.format(left))
            print('Top: ' + '{0:.0f}'.format(top))
            print('Label Width: ' + "{0:.0f}".format(width))
            print('Label Height: ' + "{0:.0f}".format(height))

            points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top))
            draw.line(points, fill='#00d400', width=5)

    image.show()

In [5]:
def show_custom_labels(model,bucket,photo, min_confidence):
    client=boto3.client('rekognition')

    #Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)

    # For object detection use case, uncomment below code to display image.
    # display_image(bucket,photo,response)

    return len(response['CustomLabels'])

In [1]:
import time 

client=boto3.client('rekognition')

bucket='test-rekog-unit'
photo='Tire_KumhoLogo_180_1.jpg'
model='arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101'
min_confidence=95

start_time = time.time()

response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
    MinConfidence=min_confidence,
    ProjectVersionArn=model)

print("-- %s seconds ---"%(time.time() - start_time))

print(response)

NameError: name 'boto3' is not defined

In [4]:
import boto3
import time 

client=boto3.client('rekognition')

bucket='test-rekog-unit'
photo='Tire_KumhoLogo_180_1.jpg'
model='arn:aws:rekognition:us-east-1:216064490951:project/kumho-logo/version/kumho-logo.2021-09-05T22.55.46/1630850147101'
min_confidence=95

start_time = time.time()

response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
    MinConfidence=min_confidence,
    ProjectVersionArn=model)

print("-- %s seconds ---"%(time.time() - start_time))

print(response)

-- 3.4239370822906494 seconds ---
{'CustomLabels': [{'Name': 'kumho', 'Confidence': 98.03800201416016, 'Geometry': {'BoundingBox': {'Width': 0.5634300112724304, 'Height': 0.17958000302314758, 'Left': 0.18614999949932098, 'Top': 0.059209998697042465}}}, {'Name': 'kumho', 'Confidence': 97.23699951171875, 'Geometry': {'BoundingBox': {'Width': 0.5221899747848511, 'Height': 0.146479994058609, 'Left': 0.26096999645233154, 'Top': 0.7740399837493896}}}], 'ResponseMetadata': {'RequestId': 'c95c998f-82cd-4e9a-be66-a26e52dbcf4a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sat, 02 Oct 2021 08:17:26 GMT', 'x-amzn-requestid': 'c95c998f-82cd-4e9a-be66-a26e52dbcf4a', 'content-length': '388', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}
